In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import desc
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import round



In [0]:
spark = SparkSession.builder.appName('get it').getOrCreate()

In [0]:
df_product = spark.table('default.dim_products_7_csv')
df_orders = spark.table('default.fact_orders_6_csv')

In [0]:
display(df_orders.limit(10))

customer_id,order_id,order_item_id,seller_id,product_id,order_purchase_date,order_approved_date,order_delivered_carrier_date,order_delivered_customer_date,order_status,price_in_cad,freight_value_in_cad,payment_type,payment_sequential,payment_installments
"CUST-100-104-bhf""",ORD-301-204-hay#,1,SELL-418-949-jbn$,PROD-351-803-btb.,2019-01-10,2019-01-10,2019-01-18,2019-01-24,delivered,1.42,0.1,credit_card,1.0,8.0
"CUST-100-104-bhf""",ORD-301-204-hay#,2,SELL-418-949-jbn$,PROD-592-252-fgo*,2019-06-10,2019-06-10,2019-06-18,2019-06-24,delivered,12.01,0.84,credit_card,1.0,8.0
CUST-100-123-fcl),ORD-286-990-fbe!,1,"SELL-659-656-lqi""",PROD-981-360-tsx.,2018-05-14,2018-05-15,2018-05-16,2018-06-03,delivered,17.28,1.21,credit_card,1.0,3.0
CUST-100-126-tie#,"ORD-986-108-ixj""",1,SELL-919-776-rtr!,PROD-466-709-ruw*,2018-12-15,2018-12-15,2018-12-16,2018-12-19,delivered,30.69,2.15,credit_card,1.0,1.0
CUST-100-146-hpj%,"ORD-933-585-zin""",1,SELL-408-580-yfc$,PROD-540-989-oyk-,2018-09-22,2018-09-22,2018-09-22,2018-09-24,delivered,40.15,2.81,credit_card,1.0,1.0
CUST-100-146-hpj%,"ORD-933-585-zin""",2,SELL-408-580-yfc$,PROD-540-989-oyk-,2019-08-22,2019-08-22,2019-08-22,2019-08-24,delivered,12.1,0.85,credit_card,1.0,1.0
CUST-100-168-kch*,ORD-220-120-epg!,1,SELL-910-309-nvm',PROD-459-947-tqu%,2018-08-11,2018-08-12,2018-08-12,2262-08-11,shipped,18.53,1.3,credit_card,1.0,4.0
CUST-100-172-blo),"ORD-850-867-nse,",1,"SELL-362-916-bpg,",PROD-264-409-eez*,2017-07-01,2017-07-01,2017-07-02,2017-07-12,delivered,20.83,1.46,credit_card,1.0,1.0
CUST-100-217-mjd$,ORD-796-367-ubx$,1,SELL-203-875-fjq),PROD-751-767-isp',2018-10-02,2018-10-02,2018-10-05,2018-10-18,delivered,21.17,1.48,credit_card,1.0,8.0
CUST-100-226-gcx+,ORD-598-520-mwf(,1,SELL-587-959-rnq&,PROD-775-222-mcv.,2018-12-28,2018-12-28,2018-12-29,2018-12-30,delivered,39.83,2.79,credit_card,1.0,2.0


In [0]:
df_orders = (
    df_orders.withColumn('order_item_id', df_orders['order_item_id'].cast(IntegerType()))
    .withColumn('order_purchase_date', f.col('order_purchase_date').cast('date'))
    .withColumn('order_approved_date', f.col('order_approved_date').cast('date'))
    .withColumn('order_delivered_carrier_date', f.col('order_delivered_carrier_date').cast('date'))
    .withColumn('order_delivered_customer_date', f.col('order_delivered_customer_date').cast('date'))
    .withColumn('price_in_cad', df_orders['price_in_cad'].cast(IntegerType()))
    .withColumn('freight_value_in_cad', df_orders['freight_value_in_cad'].cast(IntegerType()))
    .withColumn('payment_sequential', f.col('payment_sequential').cast('float'))
    )
    


In [0]:
df_orders_payment = (
    df_orders.groupBy('payment_type').agg(f.sum('price_in_cad').alias('total_price_in_cad')).orderBy(desc('total_price_in_cad'))
)
display(df_orders_payment)

payment_type,total_price_in_cad
credit_card,1435682
bank draft,375043
voucher,48893
debit_card,27322


Databricks visualization. Run in Databricks to view.

In [0]:
df_orders_payment_avg = (
    df_orders_payment.groupBy('payment_type').agg(f.avg('total_price_in_cad').alias('avg_price_in_cad'))
)

total_df_orders_payment = df_orders_payment_avg.agg(f.sum('avg_price_in_cad').alias('total_avg')).collect()[0]['total_avg']

df_orders_payment_avg = df_orders_payment_avg.withColumn('percentagem', (f.col('avg_price_in_cad') / total_df_orders_payment) * 100)

df_orders_payment_avg = df_orders_payment_avg.withColumn('percent_%', round(f.col('percentagem'),2))

df_orders_payment_avg = df_orders_payment_avg.drop('percentagem')

display(df_orders_payment_avg)

payment_type,avg_price_in_cad,percent_%
credit_card,1435682.0,76.09
bank draft,375043.0,19.88
voucher,48893.0,2.59
debit_card,27322.0,1.45


Databricks visualization. Run in Databricks to view.

In [0]:
display(df_product.limit(10))

product_id,prod_cat_name_eng
PROD-351-803-btb.,health_beauty
PROD-592-252-fgo*,health_beauty
PROD-981-360-tsx.,health_beauty
PROD-466-709-ruw*,computers_accessories
PROD-540-989-oyk-,computers_accessories
PROD-459-947-tqu%,furniture_decor
PROD-264-409-eez*,telephony
PROD-751-767-isp',sports_leisure
PROD-775-222-mcv.,computers_accessories
PROD-451-404-zij$,furniture_decor


In [0]:
products_price = df_product.join(df_orders, on='product_id', how='inner')
products_price = products_price.select('product_id', 'prod_cat_name_eng', 'price_in_cad' ).groupBy('prod_cat_name_eng').agg(f.sum('price_in_cad').alias('total_price_in_cad')).orderBy(desc('total_price_in_cad'))
display(products_price)

prod_cat_name_eng,total_price_in_cad
health_beauty,284386
watches_gifts,275455
bed_bath_table,273124
sports_leisure,227735
computers_accessories,205544
furniture_decor,162454
housewares,140904
auto,133394
toys,109310
telephony,74634


Databricks visualization. Run in Databricks to view.